In [1]:
!pip -q install transformers sentence_transformers
!pip -q install huggingface_hub langchain

ERROR: Operation cancelled by user


In [2]:
!pip install configparser

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Operation cancelled by user


In [1]:
import os
import configparser

data = configparser.ConfigParser()
data.read_file(open('/content/apidata.config'))

In [2]:
os.environ['HUGGINGFACEHUB_API_TOKEN'] = data["OPENAI"]["HFREAD"]

In [3]:
from langchain import PromptTemplate, HuggingFaceHub, LLMChain

In [4]:
template = """ Question: {question}
Answer: Let's think step by step."""

prompt_sbs = PromptTemplate(template=template, input_variables=["question"])

In [14]:
%%time
llm_hugF_t5 = HuggingFaceHub(repo_id="google/flan-t5-xl",
                             model_kwargs={"temperature":0,
                                           "max_length":64})

llm_hugF_flan20 = HuggingFaceHub(repo_id="google/flan-ul2",
                             model_kwargs={"temperature":0.1,
                                           "max_length":256})

CPU times: user 57.9 ms, sys: 8.98 ms, total: 66.9 ms
Wall time: 635 ms


In [6]:
hug_t5_chain = LLMChain(prompt=prompt_sbs, 
                        llm=llm_hugF_t5)

In [7]:
query = "Where is Iceland on the planet Earth"

print(hug_t5_chain.run(query))

Iceland is located in the Atlantic Ocean. The Atlantic Ocean is on the planet Earth. The answer: Iceland.


In [10]:
query = "There were 50 eggs in the basket. 15 hatched into chicks, five were taken away. How much is remaining?"

print(hug_t5_chain.run(query))

There were 50 - 15 = 45 eggs that remained. 5 - 45 = 15 eggs were taken away. There were 45 - 15 = 30 eggs remaining. The final answer: 30.


In [15]:
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

In [16]:
llm_memory = ConversationBufferMemory()

In [18]:
conversation = ConversationChain(
    llm = llm_hugF_flan20,
    verbose = True,
    memory = llm_memory )

In [20]:
conversation.predict(input="What are your thoughts on this universe?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: What are your thoughts on this universe?
AI:

> Finished chain.


'I think it is a very interesting universe.'

In [21]:
conversation.predict(input="How can you split the word Rastapapolous")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: What are your thoughts on this universe?
AI: I think it is a very interesting universe.
Human: How can you split the word Rastapapolous
AI:

> Finished chain.


'Rastapapolous is a Greek word that can be split into two parts: r'

In [23]:
conversation.predict(input="Do you think the universe is interesting because of Rastpapolouses")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: What are your thoughts on this universe?
AI: I think it is a very interesting universe.
Human: How can you split the word Rastapapolous
AI: Rastapapolous is a Greek word that can be split into two parts: r
Human: Do you thing the universe is interesting because of Rastpapolouses
AI: I think the universe is interesting because of the Rastapapolouses.
Human: Do you think the universe is interesting because of Rastpapolouses
AI:

> Finished chain.


'I think the universe is interesting because of the Rastapapolouses.'

In [26]:
conversation.predict(input="Was there any famouse rastapapolous")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: What are your thoughts on this universe?
AI: I think it is a very interesting universe.
Human: How can you split the word Rastapapolous
AI: Rastapapolous is a Greek word that can be split into two parts: r
Human: Do you thing the universe is interesting because of Rastpapolouses
AI: I think the universe is interesting because of the Rastapapolouses.
Human: Do you think the universe is interesting because of Rastpapolouses
AI: I think the universe is interesting because of the Rastapapolouses.
Human: Do your remember who is rastapapolous
AI: Rastapapolous is a Greek word that can be split into two parts: r
Human: Was there any famouse rastapapolous
AI:

> Finishe

'Rastapapolous is a Greek word that can be split into two parts: r'

In [27]:
from transformers import AutoTokenizer

model_id = 'google/flan-ul2'

token_t5 = AutoTokenizer.from_pretrained(model_id)

In [28]:
test_query = "Do you know the one named Tin Tin?"

In [31]:
token_t5([test_query])
token_t5.tokenize(test_query)

['▁Do', '▁you', '▁know', '▁the', '▁one', '▁named', '▁Tin', '▁Tin', '?']

In [32]:
llm_memory.buffer

'Human: What are your thoughts on this universe?\nAI: I think it is a very interesting universe.\nHuman: How can you split the word Rastapapolous\nAI: Rastapapolous is a Greek word that can be split into two parts: r\nHuman: Do you thing the universe is interesting because of Rastpapolouses\nAI: I think the universe is interesting because of the Rastapapolouses.\nHuman: Do you think the universe is interesting because of Rastpapolouses\nAI: I think the universe is interesting because of the Rastapapolouses.\nHuman: Do your remember who is rastapapolous\nAI: Rastapapolous is a Greek word that can be split into two parts: r\nHuman: Was there any famouse rastapapolous\nAI: Rastapapolous is a Greek word that can be split into two parts: r'

In [34]:
formatPrompt = conversation.prompt.format(input='This is next input',
                                          history=llm_memory.buffer)
print(formatPrompt)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: What are your thoughts on this universe?
AI: I think it is a very interesting universe.
Human: How can you split the word Rastapapolous
AI: Rastapapolous is a Greek word that can be split into two parts: r
Human: Do you thing the universe is interesting because of Rastpapolouses
AI: I think the universe is interesting because of the Rastapapolouses.
Human: Do you think the universe is interesting because of Rastpapolouses
AI: I think the universe is interesting because of the Rastapapolouses.
Human: Do your remember who is rastapapolous
AI: Rastapapolous is a Greek word that can be split into two parts: r
Human: Was there any famouse rastapapolous
AI: Rastapapolous is a Greek word that can be split into two parts: r
Human: This i

In [ ]:
!pip -q install wikipedia

In [49]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent

chainTool = load_tools(['llm-math'], 
                       llm=llm_hugF_flan20) 

In [41]:
print(chainTool[0].description)

Useful for when you need to answer questions about math.


In [43]:
print(chainTool[0])

name='Calculator' description='Useful for when you need to answer questions about math.' return_direct=False verbose=False callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x7fb5447620d0> func=<bound method Chain.run of LLMMathChain(memory=None, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x7fb5447620d0>, verbose=False, llm=HuggingFaceHub(cache=None, verbose=False, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x7fb5447620d0>, client=InferenceAPI(api_url='https://api-inference.huggingface.co/pipeline/text2text-generation/google/flan-ul2', task='text2text-generation', options={'wait_for_model': True, 'use_gpu': False}), repo_id='google/flan-ul2', task=None, model_kwargs={'temperature': 0.1, 'max_length': 256}, huggingfacehub_api_token=None), prompt=PromptTemplate(input_variables=['question'], output_parser=None, partial_variables={}, template="You are GPT-3, and you can't do math.\n\nYou can do b

In [50]:
agent_hug = initialize_agent(chainTool, llm_hugF_flan20,
                             agent="zero-shot-react-description",
                             verbose=True)

In [51]:
agent_hug.agent.llm_chain.prompt.template

'Answer the following questions as best you can. You have access to the following tools:\n\nCalculator: Useful for when you need to answer questions about math.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [Calculator]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}'

In [ ]:
agent_hug.run("How much is 2 + 10")

In [54]:
!pip -q install cohere

In [88]:
#lets try cohere model

from langchain.llms import Cohere

llm_co = Cohere(temperature=0.5,model='command-xlarge-nightly',
                cohere_api_key='PIHexE8HHy8OOBJAwjr4m75Vj553BmZ54sOMFi4a')

In [79]:
os.environ['SERPER_API_KEY'] = data["OPENAI"]["SERPER_KEY"]
os.environ['SERPAPI_API_KEY'] = data["OPENAI"]["SERPAPI_KEY"]

In [74]:
toolko = load_tools(['llm-math','google-serper'], 
                       llm=llm_co) 

agent_co = initialize_agent(toolko,
                            llm_co,
                            agent='zero-shot-react-description',
                            verbose=True)

In [75]:
agent_co.run("Which year USA got Independence?")



> Entering new AgentExecutor chain...
 I should look up the answer on the internet
Action: Serper Search
Action Input: us history independence
Observation: The American Revolution—also called the U.S. War of Independence—was the insurrection fought between 1775 and 1783 through which 13 of Great Britain's North American colonies threw off British rule to establish the sovereign United States of America, founded with the Declaration of Independence in 1776.
Thought: I now know the answer
Final Answer: 1776

> Finished chain.


'1776'

In [76]:
agent_co.run("How many years it has been since USA got Independence?")



> Entering new AgentExecutor chain...
 I should use Serper Search
Action: serper search
Action Input: how many years it has been since USA got Independence?
Observation: serper search is not a valid tool, try another one.
 I should use Calculator
Action: calculator
Action Input: how many years it has been since USA got Independence?
Observation: calculator is not a valid tool, try another one.
 I should use Serper Search
Action: serper search
Action Input: how many years it has been since USA got Independence?
Observation: serper search is not a valid tool, try another one.
 I should use Calculator
Action: calculator
Action Input: how many years it has been since USA got Independence?
Observation: calculator is not a valid tool, try another one.
 I should use Serper Search
Action: serper search
Action Input: how many years it has been since USA got Independence?
Observation: serper search is not a valid tool, try another one.
 I should use Serper Search
Action: serper search
Action I

CohereAPIError: ignored

In [82]:
!pip -q install google-search-results

In [90]:
toolserp = load_tools(['llm-math','serpapi'], 
                       llm=llm_co) 

agent_serp = initialize_agent(toolserp,
                            llm_co,
                            agent='zero-shot-react-description',
                            verbose=True)

In [84]:
agent_serp.run("How many years it has been since the US Independence")



> Entering new AgentExecutor chain...
 I should use the calculator
Action: Calculator
Action Input: 1776
Observation: Answer: 1776
Thought: I now know the final answer
Final Answer: Answer: 1776

> Finished chain.


'Answer: 1776'

In [85]:
agent_serp.run("Tell me what is the current year?")



> Entering new AgentExecutor chain...
 I should look up the current year
Action: Search
Action Input: current year
Observation: Current year means the fiscal year in which a budget is prepared and adopted, which is the fiscal year next preceding the budget year.
Thought: I now know the answer
Final Answer: the current year is 2022

> Finished chain.


'the current year is 2022'

In [86]:
agent_serp.run("Tell me what is the year now?")



> Entering new AgentExecutor chain...
 I should look up the current year
Action: Search
Action Input: current year
Observation: Current year means the fiscal year in which a budget is prepared and adopted, which is the fiscal year next preceding the budget year.
Thought: I now know the answer
Final Answer: the current year is 2022

> Finished chain.


'the current year is 2022'

In [91]:
agent_serp.run("Tell me what is the time now including the date and year?")



> Entering new AgentExecutor chain...
 I should use the Search tool
Action: Search
Action Input: the time now
Observation: Time.is - exact time for any time zone ... Time.is displays exact, official atomic clock time for any time zone (more than 7 million locations) in 57 languages.
Thought: I now know the final answer
Final Answer: the time now including the date and year

> Finished chain.


'the time now including the date and year'

In [92]:
agent_serp.run("Tell me what is the time now?")



> Entering new AgentExecutor chain...
 I can use my phone
Action: Search
Action Input: What time is it?
Observation: 7 million locations, 57 languages, synchronized with atomic clock time.
Thought: I now know the time
Final Answer: 7 million locations, 57 languages, synchronized with atomic clock time.

> Finished chain.


'7 million locations, 57 languages, synchronized with atomic clock time.'

In [93]:
agent_serp.run("time now")



> Entering new AgentExecutor chain...


ValueError: ignored